In [1]:
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import re

In [ ]:
# Effacer un projet
# bd.projects.delete_project('DOC_casestudies', delete_dir=True)

In [ ]:
# Supprimer la base de données
# del bd.databases['ecoinvent-3.10-cutoff']

In [4]:
bd.projects.set_current(name='DOC_casestudies')

In [5]:
list(bd.projects)

[Project: default, Project: DOC_casestudies]

In [ ]:
# Importer ecoinvent
#bi.import_ecoinvent_release(
#    version='3.10',
#    system_model='cutoff',
#    username='a',
#    password='A'
#)

In [9]:
# Créer une base de données :
#elec_bl = bd.Database('elec_bl').register()

In [6]:
list(bd.databases)

['ecoinvent-3.10-biosphere', 'ecoinvent-3.10-cutoff', 'elec_bl']

In [7]:
eidb = bd.Database("ecoinvent-3.10-cutoff")

In [8]:
elec_bl = bd.Database("elec_bl")

In [62]:
# Lister toutes les activités dans ma bdd
for activity in elec_bl.search('*'):
    print(f"{activity['name']}, {activity.key[1]}")

In [61]:
# Supprimer toutes les activités de la base de données
for activity in elec_bl:
    activity.delete()

In [12]:
# Pour filtrer parmi les activités eidb
from collections import Counter

In [63]:
# Récupérer les activités depuis eidb selon mon excel et les stocker dans un dictionnaire

# Charger le fichier Excel
df = pd.read_excel('process_names.xlsx', header=None)

# Dictionnaire pour stocker les activités trouvées
activities = {}

# Liste pour stocker les processus non trouvés
not_found = []

# Liste des mots à enlever
words_to_remove = {'processing', 'high voltage', 'low voltage', 'central or small-scale', 'other than natural gas', 'photovoltaic plant'}

# Fonction pour extraire et diviser les chaînes de caractères
def extract_strings(s):
    match = re.match(r'([^{}]+)\{([^{}]+)\}([^{}]+)?', s)
    if match:
        name_part = match.group(1).strip().lower()
        location_part = match.group(2).strip()
        other_part = match.group(3).strip().lower() if match.group(3) else ""
        
        # Diviser la première partie à chaque virgule et enlever les espaces autour
        name_parts = [name.strip() for name in name_part.split(',')]
        # Enlever les caractères '| ' et diviser la troisième partie à chaque virgule
        other_parts = [part.strip().replace('| ', '') for part in other_part.split(',')]
        
        return name_parts, location_part, other_parts
    return None, None, None

# Fonction pour vérifier si une sous-chaîne contient un des mots à enlever
def contains_any_word(substring, words):
    substring = substring.lower()
    return any(word in substring for word in words)

# Fonction pour nettoyer les sous-chaînes
def filter_substrings(substrings, words_to_remove):
    return [sub for sub in substrings if not contains_any_word(sub, words_to_remove)]

# Fonction pour calculer la similarité en fonction du nombre de mots communs
def calculate_similarity(excel_parts, activity_name):
    activity_words = activity_name.lower().split()
    common_words = set(excel_parts).intersection(activity_words)
    return len(common_words), abs(len(activity_words) - len(excel_parts))

# Fonction pour trouver l'activité qui partage le plus de mots
def find_best_match(matching_activities, excel_parts):
    similarities = [
        (act, calculate_similarity(excel_parts, act['name']))
        for act in matching_activities
    ]
    
    # Trier en fonction du nombre de mots partagés et du nombre de mots les plus proches
    similarities.sort(key=lambda x: (x[1][0], -x[1][1]), reverse=True)
    
    # Retourner l'activité la plus similaire
    return similarities[0][0] if similarities else None

# Parcourir chaque ligne du fichier Excel
for index, row in df.iterrows():
    nom_perso = row[0]  # Le nom que tu souhaites donner à l'activité
    nom_eco = row[1]    # Le nom de l'activité dans ecoinvent
    
    # Extraire les chaînes de caractères
    name_parts, location_part, other_parts = extract_strings(nom_eco)
    
    if name_parts and location_part and other_parts:
        # Nettoyer les parties du nom
        cleaned_name_parts = filter_substrings(name_parts, words_to_remove)
        cleaned_other_parts = filter_substrings(other_parts, words_to_remove)
        
        # Enlever les doublons entre les parties du nom
        cleaned_name_parts = list(set(cleaned_name_parts) - set(cleaned_other_parts))
        
        # Recherche des activités correspondant à toutes les sous-chaînes du premier et troisième groupe et à la localisation
        matching_activities = [
            act for act in eidb
            if all(name in act['name'].lower() for name in cleaned_name_parts) and
               location_part == act['location'] and
            (not cleaned_other_parts or all(part in act['name'].lower() for part in cleaned_other_parts))
        ]
        
        if matching_activities:
            # Trouver l'activité qui partage le plus de mots
            best_match = find_best_match(matching_activities, cleaned_name_parts)
            
            if best_match:
                activities[nom_perso] = best_match
            else:
                activities[nom_perso] = matching_activities[0]  # Utiliser la première si aucune meilleure correspondance trouvée
        else:
            # Si aucune activité ne correspond, ajouter à la liste not_found
            not_found.append(nom_eco)
    else:
        # Si la chaîne de caractères n'est pas conforme, ajouter à la liste not_found
        not_found.append(nom_eco)

# Afficher les activités trouvées
for nom_perso, activity in activities.items():
    print(f"{nom_perso}: {activity['name']} ({activity['location']})")

# Signaler les processus non trouvés
if not_found:
    print("\nLes processus suivants n'ont pas été trouvés dans ecoinvent:")
    for process in not_found:
        print(f"- {process}")
else:
    print("\nTous les processus ont été trouvés avec succès.")
    len(activities)

act_build_machine: diesel, burned in building machine (GLO)
dist_network: distribution network construction, electricity, low voltage (CA-QC)
elec_gen_set_10MW: diesel, burned in diesel-electric generating set, 10MW (GLO)
elec_pv_3kWp: electricity production, photovoltaic, 3kWp facade installation, multi-Si, panel, mounted (RoW)
excav_digger: excavation, hydraulic digger (RoW)
excav_skidsteer: excavation, skid-steer loader (RoW)
heat_oil_boiler: heat production, light fuel oil, at boiler 100kW, non-modulating (RoW)
heat_pellet_boiler: heat production, wood pellet, at furnace 300kW (RoW)
maint_lorry: maintenance, lorry 16 metric ton (RoW)
maint_pplant: electricity production, oil (CA-NU)
maint_road: road maintenance (RoW)
oil_boiler: oil boiler production, 100kW (RoW)
oil_boiler_chimney: chimney production (RoW)
oil_storage: oil storage production, 3000l (RoW)
pellet_boiler: furnace production, pellets, with silo, 300kW (RoW)
power_plant: oil power plant construction, 500MW (RoW)
pv_cab

In [14]:
import uuid

# Générateur d'ID unique
def generate_unique_id(prefix=""):
    return f"{prefix}{uuid.uuid4().hex[:8]}"

In [64]:
# Diviser les activités en onsite, offsite et waste, puis attribuer les exchanges correspondants

for activity_name in activities:
    #print(activity_name)
    activity = activities.get(activity_name)
    
    # Offsite activity
    offsite_activity_data = {
        'name': 'offsite_' + activity['name'],
        'unit': activity['unit'],
        'exchanges': []
    }
    
    # Onsite activity
    onsite_activity_data = {
        'name': 'onsite_' + activity['name'],
        'unit': activity['unit'],
        'exchanges': []
    }
    
    # Waste activity
    waste_activity_data = {
        'name': 'waste_' + activity['name'],
        'unit': activity['unit'],
        'exchanges': []
    }
    
    # Créer et sauvegarder les activités
    offsite_activity = elec_bl.new_activity(code=generate_unique_id('off_'), **offsite_activity_data)
    onsite_activity = elec_bl.new_activity(code=generate_unique_id('on_'), **onsite_activity_data)
    waste_activity = elec_bl.new_activity(code=generate_unique_id('waste_'), **waste_activity_data)
    
    offsite_activity.save()
    onsite_activity.save()
    waste_activity.save()

    
    # Ajouter les échanges à l'activité appropriée
    for exc in activity.technosphere():
        # Déchet vers waste
        if exc.amount < 0:
            waste_activity.new_exchange(
                input=exc.input,
                amount=exc.amount,
                type=exc['type'],
                unit=exc['unit']
            ).save()
        # Conso d'eau vers onsite
        elif 'water' in exc['name'] and exc.amount > 0:
            onsite_activity.new_exchange(
                input=exc.input,
                amount=exc.amount,
                type=exc['type'],
                unit=exc['unit']
            ).save()
        # Conso d'énergie vers onsite & offsite
        elif exc['unit'] in ['kilowatt hour', 'megajoule']:
            onsite_activity.new_exchange(
                input=exc.input,
                amount=exc.amount,
                type=exc['type'],
                unit=exc['unit']
            ).save()
            offsite_activity.new_exchange(
                input=exc.input,
                amount=exc.amount,
                type=exc['type'],
                unit=exc['unit']
            ).save()
        # Process multiregionaux vers onsite & offsite
        elif ('excavation' in exc['name'] or 'diesel, burned in building machine' in exc['name'] or 'lorry' in exc['name'] or 'road' in exc['name']) and exc.amount > 0:
            onsite_activity.new_exchange(
                input=exc.input,
                amount=exc.amount,
                type=exc['type'],
                unit=exc['unit']
            ).save()
            offsite_activity.new_exchange(
                input=exc.input,
                amount=exc.amount,
                type=exc['type'],
                unit=exc['unit']
            ).save()
        # Autres techno vers offsite
        else:
            offsite_activity.new_exchange(
                input=exc.input,
                amount=exc.amount,
                type=exc['type'],
                unit=exc['unit']
            ).save()

    
    # Ajouter les échanges biosphériques à l'activité onsite
    for exc in activity.biosphere():
        onsite_activity.new_exchange(
            input=exc.input,
            amount=exc.amount,
            type=exc['type'],
            unit=exc['unit']
        ).save()
    print(activity['name'] + ' ok')

len(elec_bl)

diesel, burned in building machine ok
distribution network construction, electricity, low voltage ok
diesel, burned in diesel-electric generating set, 10MW ok
electricity production, photovoltaic, 3kWp facade installation, multi-Si, panel, mounted ok
excavation, hydraulic digger ok
excavation, skid-steer loader ok
heat production, light fuel oil, at boiler 100kW, non-modulating ok
heat production, wood pellet, at furnace 300kW ok
maintenance, lorry 16 metric ton ok
electricity production, oil ok
road maintenance ok
oil boiler production, 100kW ok
chimney production ok
oil storage production, 3000l ok
furnace production, pellets, with silo, 300kW ok
oil power plant construction, 500MW ok
photovoltaics, electric installation for 3kWp module, at building ok
photovoltaic facade installation, 3kWp, multi-Si, panel, mounted, at building ok
inverter production, 2.5kW ok
photovoltaic mounting system production, for facade installation ok
photovoltaic panel production, multi-Si wafer ok
road co

87

In [65]:
# Additionner les échanges avec le même nom dans le RoW ou le GLO

# Parcourir chaque activité dans la base de données
for activity in elec_bl:
    # Dictionnaire pour stocker les échanges par nom
    exchanges_by_name = {}

    # Itérer à travers les échanges de l'activité
    for exc in activity.technosphere():
        name = exc.input.get('name')  # Utiliser .get() pour éviter KeyError
        
        if name is None:
            continue  # Passer à l'échange suivant si le nom est absent
        
        # Ajouter l'échange à la liste correspondante à ce nom
        if name not in exchanges_by_name:
            exchanges_by_name[name] = []
        exchanges_by_name[name].append(exc)

    # Parcourir chaque groupe d'échanges par nom
    for name, exchanges in exchanges_by_name.items():
        if len(exchanges) > 1:  # Si plusieurs échanges avec le même nom
            row_exchange = None
            total_amount = 0

            for exc in exchanges:
                total_amount += exc.get('amount', 0)  # Utiliser .get() pour éviter KeyError

                if exc.input.get('location') == 'RoW' or exc.input.get('location') == 'GLO':
                    row_exchange = exc  # Garder l'échange avec location 'RoW'

            if row_exchange:
                row_exchange['amount'] = total_amount  # Mettre à jour le montant
                row_exchange.save()
                print(f"Activity '{activity['name']}' - Summing exchanges '{name}' to location '{row_exchange.input.get('location')}'. New amount: {total_amount}")

            # Supprimer les autres échanges avec ce nom
            for exc in exchanges:
                if exc != row_exchange:
                    exc.delete()

    activity.save()
len(elec_bl)

Activity 'offsite_photovoltaic panel production, multi-Si wafer' - Summing exchanges 'market for corrugated board box' to location 'RoW'. New amount: 1.0956
Activity 'offsite_photovoltaic panel production, multi-Si wafer' - Summing exchanges 'market for heat, district or industrial, natural gas' to location 'RoW'. New amount: 4.86639
Activity 'offsite_photovoltaic panel production, multi-Si wafer' - Summing exchanges 'market for methanol' to location 'RoW'. New amount: 0.0021556
Activity 'waste_heat production, wood pellet, at furnace 300kW' - Summing exchanges 'market for wood ash mixture, pure' to location 'RoW'. New amount: -0.00031733333333333396
Activity 'offsite_maintenance, lorry 16 metric ton' - Summing exchanges 'market for heat, district or industrial, other than natural gas' to location 'RoW'. New amount: 53864.99999999999
Activity 'waste_inverter production, 2.5kW' - Summing exchanges 'market for waste polyethylene' to location 'RoW'. New amount: -0.05999999999999999
Activi

87

In [66]:
# Remplacer les exchanges des waste par ceux présents dans ma bdd

# Table de correspondance des mots clés et des processus cibles
keyword_to_process = {
    'municipal solid waste': 'waste_market for municipal solid waste',
    'plastic, mixture': 'waste_market for waste plastic, mixture',
    'polystyrene': 'waste_market for waste polystyrene',
    'polyvinylchloride': 'waste_market for waste polyvinylchloride',
    'polyethylene': 'waste_market for waste polyethylene'
}

# Step 1: Find all activities with 'waste_' in their name
waste_activities = [act for act in elec_bl if 'waste_' in act['name']]

# Step 2: Define fallback process ('waste_market for municipal solid waste')
fallback_process_name = 'waste_market for municipal solid waste'
fallback_process = [act for act in elec_bl if fallback_process_name in act['name']][0]

# Step 3: Create a list of activities from the correspondence table to exclude from processing
excluded_activities = set(keyword_to_process.values())  # Extracts the process names from the correspondence table

# Step 4: Process each activity's exchanges, excluding those in the correspondence table
for activity in waste_activities:
    # Skip the activity if it's in the excluded activities list
    if activity['name'] in excluded_activities:
#        print(f"Skipping activity '{activity['name']}' as it is in the correspondence table.")
        continue
    for exc in activity.exchanges():
        # Check if the exchange's input name contains any of the keywords
        for keyword, new_process in keyword_to_process.items():
            if keyword in exc.input['name']:
                # Replace the exchange's input with the new process
                new_input_activity = [act for act in elec_bl if new_process in act['name']][0]  # Find the new process
                
                exc['input'] = new_input_activity.key  # Met à jour la référence de l'échange
                exc.save()  # Enregistre l'échange mis à jour
                print(f"Replaced '{exc.input['name']}' with '{new_process}'")
            elif 'wastewater' in exc.input['name']:
                print('Did not replace wastewater')
                continue
            else:
                # If no keyword was matched, replace the exchange with the fallback process
                exc['input']=fallback_process.key
                exc.save()  # Enregistre l'échange mis à jour
                print(f"Replaced '{exc.input['name']}' with '{fallback_process_name}'")
    
        # Save changes after each exchange
#        activity.save()
#        print(f"Saved changes for exchange in activity '{activity['name']}'")

    # Optionally, save the activity again after processing all exchanges
    activity.save()
    print(f"Saved changes for activity '{activity['name']}'")

# Optional: Save changes for the entire database if needed
# db.save()
len(elec_bl)

Replaced 'market for waste concrete' with 'waste_market for municipal solid waste'
Replaced 'market for waste concrete' with 'waste_market for municipal solid waste'
Replaced 'market for waste concrete' with 'waste_market for municipal solid waste'
Replaced 'market for waste concrete' with 'waste_market for municipal solid waste'
Replaced 'market for waste concrete' with 'waste_market for municipal solid waste'
Replaced 'market for waste mineral wool, for final disposal' with 'waste_market for municipal solid waste'
Replaced 'market for waste mineral wool, for final disposal' with 'waste_market for municipal solid waste'
Replaced 'market for waste mineral wool, for final disposal' with 'waste_market for municipal solid waste'
Replaced 'market for waste mineral wool, for final disposal' with 'waste_market for municipal solid waste'
Replaced 'market for waste mineral wool, for final disposal' with 'waste_market for municipal solid waste'
Saved changes for activity 'waste_chimney producti

87

In [67]:
# Garder seulement les open dump, wet infiltration class

for activity in waste_activities:
    # Skip the activity if it's in the excluded activities list
    if activity['name'] in excluded_activities:
        od500mm_exchange = None
        total_amount = 0
        for exc in activity.exchanges():
            total_amount += exc.get('amount', 0)  # Utiliser .get() pour éviter KeyError
            if 'open dump' in exc.input['name'] and '500mm' in exc.input['name']:
                    od500mm_exchange = exc  # Garder l'échange avec location 'RoW'
            if od500mm_exchange:
                od500mm_exchange['amount'] = total_amount  # Mettre à jour le montant
                od500mm_exchange.save()
                print(f"Activity '{activity['name']}' - Summing exchanges '{name}' to open dump 500mm. New amount: {total_amount}")
            if exc != od500mm_exchange:
                    exc.delete()
    activity.save()

Activity 'waste_market for waste polystyrene' - Summing exchanges 'market for transport, freight, lorry, unspecified' to open dump 500mm. New amount: -0.687397408106803
Activity 'waste_market for waste polystyrene' - Summing exchanges 'market for transport, freight, lorry, unspecified' to open dump 500mm. New amount: -0.7373918776201953
Activity 'waste_market for waste polystyrene' - Summing exchanges 'market for transport, freight, lorry, unspecified' to open dump 500mm. New amount: -0.799728800380221
Activity 'waste_market for waste polystyrene' - Summing exchanges 'market for transport, freight, lorry, unspecified' to open dump 500mm. New amount: -0.8248214985563975
Activity 'waste_market for waste polystyrene' - Summing exchanges 'market for transport, freight, lorry, unspecified' to open dump 500mm. New amount: -0.904048346020172
Activity 'waste_market for waste polystyrene' - Summing exchanges 'market for transport, freight, lorry, unspecified' to open dump 500mm. New amount: -0.

In [68]:
# Remplacer les exchanges de transport, lorry par ceux présents dans ma bdd pour onsite

# Table de correspondance des mots clés et des processus à éviter

keyword_to_process = {
    'diesel': 'market for diesel',
    'lub oil': 'market for lubricating oil',
    'wood pellet': 'market for wood pellet, measured as dry mass',
}
# Create a list of activities from the correspondence table to exclude from processing
excluded_activities = set(keyword_to_process.values())  # Extracts the process names from the correspondence table

transport_name = 'market for transport, freight, lorry, unspecified'

# Step 1: Find all activities
for activity in elec_bl:
    # Extraire la particule de l'activité actuelle
    activity_prefix = activity['name'].split('_')[0] + '_'
    activity_generic_name = activity['name'].split('_')[1]
    if activity_generic_name in excluded_activities:
#   print(f"Skipping activity '{activity['name']}' as it is in the correspondence table.")
        continue
    # Créer une liste pour stocker les remplacements de cette activité
    replacement_report[activity['name']] = []

    replacement_activity = [
        activity for activity in elec_bl
        if 'transport' in activity['name']    
        and 'lorry' in activity['name']    
        and activity_prefix in activity['name']    
    ][0]

    # Itérer à travers les échanges de l'activité actuelle
    for exc in activity.exchanges():
        old_exc_name = exc.input['name']
        new_exc_name = replacement_activity['name']
        # Extraire le nom de base de l'échange cible (sans particule)
        if 'market for transport, freight, lorry, unspecified' in exc.input['name']:
            exc['input'] = replacement_activity.key  # Met à jour la référence de l'échange
            exc.save()  # Enregistre l'échange mis à jour
            
            # Ajouter le remplacement au rapport
            replacement_report[activity['name']].append({
                'old_exchange': old_exc_name,
                'new_activity': new_exc_name
            })
        activity.save()
    
# Afficher le rapport de remplacement
for activity_name, replacements in replacement_report.items():
    if replacements:  # Afficher uniquement si des remplacements ont été effectués
        print(f"Activity: {activity_name}")
        for replacement in replacements:
            print(f"  - Replaced '{replacement['old_exchange']}' with '{replacement['new_activity']}'")


Activity: onsite_market for waste polyvinylchloride
  - Replaced 'market for transport, freight, lorry, unspecified' with 'onsite_transport, freight, lorry >32 metric ton, EURO3'
Activity: onsite_market for waste polystyrene
  - Replaced 'market for transport, freight, lorry, unspecified' with 'onsite_transport, freight, lorry >32 metric ton, EURO3'
Activity: onsite_market for waste polyethylene
  - Replaced 'market for transport, freight, lorry, unspecified' with 'onsite_transport, freight, lorry >32 metric ton, EURO3'
Activity: offsite_market for municipal solid waste
  - Replaced 'market for transport, freight, lorry, unspecified' with 'offsite_transport, freight, lorry >32 metric ton, EURO3'
Activity: offsite_market for waste polyethylene
  - Replaced 'market for transport, freight, lorry, unspecified' with 'offsite_transport, freight, lorry >32 metric ton, EURO3'
Activity: offsite_market for waste polyvinylchloride
  - Replaced 'market for transport, freight, lorry, unspecified' w

In [69]:
# Copier les exchanges des waste_ dans les onsite_ de mes process de waste spécifiques

keyword_to_process_2 = {
    'municipal solid waste': 'market for municipal solid waste',
    'plastic, mixture': 'market for waste plastic, mixture',
    'polystyrene': 'market for waste polystyrene',
    'polyvinylchloride': 'market for waste polyvinylchloride',
    'polyethylene': 'market for waste polyethylene'
}

# Step 3: Create a list of activities from the correspondence table to exclude from processing
target_activities = set(keyword_to_process_2.values())  # Extracts the process names from the correspondence table
# Créer des dictionnaires pour les activités waste_ et onsite_
waste_activities = {}
onsite_activities = {}

# Parcourir les activités pour les classifier en waste_ et onsite_
for activity in elec_bl:
    if activity['name'].startswith('waste_'):
        base_name = activity['name'][6:]  # Nom sans le préfixe waste_
        waste_activities[base_name] = activity
    elif activity['name'].startswith('onsite_'):
        base_name = activity['name'][7:]  # Nom sans le préfixe onsite_
        onsite_activities[base_name] = activity
        
for base_name, waste_activity in waste_activities.items():
    # Skip the activity if it's NOT in the excluded activities list
    if base_name in target_activities:
        # Transférer les exchanges des activités waste_ vers les activités correspondantes onsite_   
        if base_name in onsite_activities:
            onsite_activity = onsite_activities[base_name]
        
            # Ajouter les exchanges de waste_ à onsite_
            for exchange in waste_activity.exchanges():
                new_exc = onsite_activity.new_exchange(
                    amount=exchange['amount'],
                    input=exchange['input'],
                    type=exchange['type'],
                    unit=exchange['unit']
                )
                new_exc.save()    
                print(f"Un nouvel échange vers '{exchange.input['name']}' a été ajouté au processus '{onsite_activity['name']}' avec un amount de {exchange['amount']}.")

        # Sauvegarder les changements dans l'activité onsite_
#        onsite_activity.save()

Un nouvel échange vers 'treatment of waste polystyrene, open dump, wet infiltration class (500mm)' a été ajouté au processus 'onsite_market for waste polystyrene' avec un amount de -0.9999999999999998.
Un nouvel échange vers 'treatment of municipal solid waste, open dump, wet infiltration class (500mm)' a été ajouté au processus 'onsite_market for municipal solid waste' avec un amount de -1.0000000000000004.
Un nouvel échange vers 'treatment of waste plastic, mixture, open dump, wet infiltration class (500mm)' a été ajouté au processus 'onsite_market for waste plastic, mixture' avec un amount de -1.0000000000000002.
Un nouvel échange vers 'treatment of waste polyvinylchloride, open dump, wet infiltration class (500mm)' a été ajouté au processus 'onsite_market for waste polyvinylchloride' avec un amount de -0.9999999999999998.
Un nouvel échange vers 'treatment of waste polyethylene, open dump, wet infiltration class (500mm)' a été ajouté au processus 'onsite_market for waste polyethylen

In [70]:
# Mettre les activités de waste dans les activités onsite et offsite correspondantes

keyword_to_process = {
    'municipal solid waste': 'waste_market for municipal solid waste',
    'plastic, mixture': 'waste_market for waste plastic, mixture',
    'polystyrene': 'waste_market for waste polystyrene',
    'polyvinylchloride': 'waste_market for waste polyvinylchloride',
    'polyethylene': 'waste_market for waste polyethylene'
}

excluded_activities = set(keyword_to_process.values())  # Extracts the process names from the correspondence table

# Step 1: Find all activities with 'waste_' in their name
waste_activities = [act for act in elec_bl if 'waste_' in act['name']]

for waste_activity in waste_activities:
    # Skip the activity if it's in the excluded activities list
    if waste_activity['name'] not in excluded_activities:
        # Extraire le base_name sans le préfixe waste_
        base_name = waste_activity['name'].replace('waste_', '')

        # Chercher les activités onsite_ et offsite_ correspondant au même base_name
        onsite_activity = [act for act in elec_bl if act['name'] == f'onsite_{base_name}'][0]
        offsite_activity = [act for act in elec_bl if act['name'] == f'offsite_{base_name}'][0]

        # Parcourir les exchanges de waste_activity
        for exchange in waste_activity.exchanges():
            base_exc_name = exchange.input['name'].replace('waste_', '')
            if 'waste_' in exchange.input['name']:  # Vérifie si l'exchange pointe vers une activité waste_
                # Copier l'exchange dans onsite_ et remplacer waste_ par onsite_
                if onsite_activity:
                    new_input_onsite = [act for act in elec_bl if act['name'] ==f'onsite_{base_exc_name}'][0]
                    new_exc_onsite = onsite_activity.new_exchange(
                        amount=exchange['amount'],
                        input=new_input_onsite,
                        type=exchange['type'],
                        unit=exchange['unit']
                    )
                    new_exc_onsite.save()    
                print(f"Un nouvel échange vers '{new_exc_onsite.input['name']}' a été ajouté au processus '{onsite_activity['name']}' avec un amount de {new_exc_onsite['amount']}.")                
                # Copier l'exchange dans offsite_ et remplacer waste_ par offsite_
                if offsite_activity:
                    new_input_offsite = [act for act in elec_bl if act['name'] ==f'offsite_{base_exc_name}'][0]
                    new_exc_offsite = offsite_activity.new_exchange(
                        amount=exchange['amount'],
                        input=new_input_offsite,
                        type=exchange['type'],
                        unit=exchange['unit']
                    )
                    new_exc_offsite.save()    
                print(f"Un nouvel échange vers '{new_exc_offsite.input['name']}' a été ajouté au processus '{onsite_activity['name']}' avec un amount de {new_exc_offsite['amount']}.")



Un nouvel échange vers 'onsite_market for municipal solid waste' a été ajouté au processus 'onsite_diesel, burned in building machine' avec un amount de -0.0005139999999999999.
Un nouvel échange vers 'offsite_market for municipal solid waste' a été ajouté au processus 'onsite_diesel, burned in building machine' avec un amount de -0.0005139999999999999.
Un nouvel échange vers 'onsite_market for municipal solid waste' a été ajouté au processus 'onsite_oil boiler production, 100kW' avec un amount de -6.0.
Un nouvel échange vers 'offsite_market for municipal solid waste' a été ajouté au processus 'onsite_oil boiler production, 100kW' avec un amount de -6.0.
Un nouvel échange vers 'onsite_market for municipal solid waste' a été ajouté au processus 'onsite_oil boiler production, 100kW' avec un amount de -19.0.
Un nouvel échange vers 'offsite_market for municipal solid waste' a été ajouté au processus 'onsite_oil boiler production, 100kW' avec un amount de -19.0.
Un nouvel échange vers 'onsit

In [21]:
# Ajouter les distances parcourues aux consommables (diesel, oil, wood).

# Fonction pour ajouter un nouvel échange à un processus donné
def add_new_exchange(activity, exchange, quantity, exchange_type='technosphere'):
    new_exc = activity.new_exchange(
        input=exchange,  # L'activité vers laquelle l'échange est dirigé
        amount=quantity,  # La quantité allouée
        type=exchange_type  # Type d'échange (e.g., 'technosphere' ou 'biosphere')
    )
    new_exc.save()  # Sauvegarde le nouvel échange
    print(f"Un nouvel échange vers '{exchange['name']}' a été ajouté au processus '{activity['name']}' avec un amount de {quantity}.")

# Activités cibles
diesel = [
    activity for activity in elec_bl
    if 'market for diesel' in activity['name']    
    and 'offsite' in activity['name']    
][0]
lub_oil = [
    activity for activity in elec_bl
    if 'market for lubricating oil' in activity['name']    
    and 'offsite' in activity['name']    
][0]
wood_pellet = [
    activity for activity in elec_bl
    if 'market for wood pellet' in activity['name']    
    and 'offsite' in activity['name']    
][0]

# Echanges à ajouter
petroleum_ship = [
    activity for activity in eidb
    if 'market for transport, freight, sea, tanker for petroleum' in activity['name']
][0]
container_ship = [
    activity for activity in eidb
    if 'market for transport, freight, sea, container ship' in activity['name']
][0]

# Ajouter un nouvel échange dans les deux processus
add_new_exchange(diesel, petroleum_ship, quantity=3)
add_new_exchange(lub_oil, petroleum_ship, quantity=3)
add_new_exchange(wood_pellet, container_ship, quantity=3)


Un nouvel échange vers 'market for transport, freight, sea, tanker for petroleum' a été ajouté au processus 'offsite_market for diesel' avec un amount de 3.
Un nouvel échange vers 'market for transport, freight, sea, tanker for petroleum' a été ajouté au processus 'offsite_market for lubricating oil' avec un amount de 3.
Un nouvel échange vers 'market for transport, freight, sea, container ship with reefer, cooling' a été ajouté au processus 'offsite_market for wood pellet, measured as dry mass' avec un amount de 3.


In [71]:
# Créer les activités des systèmes modélisés
scenarios = ['electricity, baseline', 'electricity, alternative', 'heat, baseline', 'heat, alternative']
phases = ['production', 'distribution', 'use', 'end of life']
locations = ['onsite', 'offsite']

# Parcourir chaque combinaison de scénario et phase pour créer des activités
for scenario in scenarios:
    for location in locations:
        # Créer une nouvelle activité dans la base de données
        activity_name = f"{location}_{scenario}"
        new_activity = elec_bl.new_activity(
            code=generate_unique_id(),  # Générer un code unique
            name=activity_name,
            unit="item",  # Unité en termes d'items
            type="process",  # Type d'activité
        )
        
        # Sauvegarder l'activité
        new_activity.save()
        for phase in phases:
            # Nom de l'activité combinant le scénario et la phase
            activity_name = f"{location}_{scenario}, {phase} phase"
    
            # Créer une nouvelle activité dans la base de données
            new_activity = elec_bl.new_activity(
                code=generate_unique_id(),  # Générer un code unique
                name=activity_name,
                unit="item",  # Unité en termes d'items
                type="process",  # Type d'activité
            )
    
            # Sauvegarder l'activité
            new_activity.save()

In [30]:
### Récupérer les noms des activités dans un excel en supprimant les préfixes et en évitant les doublons
prefixes = ['offsite_', 'onsite_', 'waste_']
activity_names = [activity['name'] for activity in elec_bl]

# Supprimer les préfixes
cleaned_activity_names = []
for name in activity_names:
    for prefix in prefixes:
        if name.startswith(prefix):
            name = name[len(prefix):]
    cleaned_activity_names.append(name)

# Enlever les doublons
unique_activity_names = list(set(cleaned_activity_names))

# Créer un DataFrame avec les noms sans doublons
df = pd.DataFrame(unique_activity_names, columns=['Activity Name'])

# Écrire le DataFrame dans un fichier Excel
#df.to_excel("activities_names.xlsx", index=False)


In [73]:
# Ajouter les exchanges de ma bdd aux activités des systèmes modélisés

import pandas as pd

# Lire le fichier Excel
df = pd.read_excel("activities_names.xlsx")

# Liste des préfixes
prefixes = ['offsite_', 'onsite_']

# Fonction pour ajouter le bon préfixe
def add_prefix(activity_name):
    for prefix in prefixes:
        if prefix + activity_name in activity_names_with_prefixes:
            return prefix + activity_name
    return activity_name

# Obtenir tous les noms d'activités avec les préfixes
activity_names_with_prefixes = [activity['name'] for activity in elec_bl]

# Parcourir chaque colonne du DataFrame (excepté la première)
for prefix in prefixes:
    for parent in df.columns[1:]:
        parent_name_with_prefix = (prefix+parent)
        print(parent_name_with_prefix)
    
        # Parcourir chaque ligne pour récupérer les fils et leurs quantités
        for index, row in df.iterrows():
            fils_name = row['Activity Name']
            quantity_fils = row[parent]
        
            # Si une quantité est inscrite, ajouter l'exchange
            if not pd.isna(quantity_fils):
                fils_name_with_prefix = (prefix+fils_name)
                print(fils_name_with_prefix, quantity_fils)

                # Trouver les activités associées dans la base de données
                parent_activity = [act for act in elec_bl if act['name'] == parent_name_with_prefix][0]
                fils_activity = [act for act in elec_bl if act['name'] == fils_name_with_prefix][0]
            
                # Ajouter l'exchange dans l'activité parent
                if parent_activity and fils_activity:
                    add_new_exchange(parent_activity, fils_activity, quantity=quantity_fils)


offsite_electricity, baseline, production phase
offsite_distribution network construction, electricity, low voltage 5.517241379310345e-09
Un nouvel échange vers 'offsite_distribution network construction, electricity, low voltage' a été ajouté au processus 'offsite_electricity, baseline, production phase' avec un amount de 5.517241379310345e-09.
offsite_oil power plant construction, 500MW 2.5300000000000002e-11
Un nouvel échange vers 'offsite_oil power plant construction, 500MW' a été ajouté au processus 'offsite_electricity, baseline, production phase' avec un amount de 2.5300000000000002e-11.
offsite_electricity, baseline, distribution phase
offsite_distribution network construction, electricity, low voltage 5.517241379310345e-09
Un nouvel échange vers 'offsite_distribution network construction, electricity, low voltage' a été ajouté au processus 'offsite_electricity, baseline, distribution phase' avec un amount de 5.517241379310345e-09.
offsite_oil power plant construction, 500MW 2.

In [104]:
# Afficher les scénarios triés et les exchanges associés

phase_activities = [act for act in elec_bl if 'use phase' in act['name']]
phase_order = ['production phase', 'distribution phase', 'use phase', 'end of life phase']

def get_scenario_phase_location(activity):

    parts_undr = activity['name'].split('_')
    location = parts_undr[0]
    parts_coma = parts_undr[1].split(', ')
    scenario = parts_coma[0] + parts_coma[1]
    phase = parts_coma[2]
    return scenario, location, phase

# Trier la liste des activités
sorted_activities = sorted(
    phase_activities,
    key=lambda act: (
        get_scenario_phase_location(act)[0],  # Trier par scénario
        get_scenario_phase_location(act)[1],  # Trier par localisation
        phase_order.index(get_scenario_phase_location(act)[2])  # Trier par phase selon l'ordre défini
    )
)

# Afficher les activités triées
for activity in sorted_activities:
    print(f"Activity: {activity['name']}")
    for exc in activity.exchanges():
        print(f"  - {exc.input['name']}")
        sub_exc_act = [act for act in elec_bl if exc.input['name'] in act['name']][0]
        for sub_exc in sub_exc_act.exchanges():
            print(f"    - {sub_exc.input['name']}")
    print(' ')

Activity: offsite_electricity, alternative, use phase
  - offsite_electricity production, photovoltaic, 3kWp facade installation, multi-Si, panel, mounted_use
  - offsite_electricity, baseline_use
 
Activity: onsite_electricity, alternative, use phase
  - onsite_electricity production, photovoltaic, 3kWp facade installation, multi-Si, panel, mounted_use
    - market for tap water
    - Water
    - Energy, solar, converted
  - onsite_electricity, baseline_use
 
Activity: offsite_electricity, baseline, use phase
  - offsite_diesel, burned in diesel-electric generating set, 10MW_use
    - market group for diesel
    - market for lubricating oil
    - offsite_market for municipal solid waste
  - offsite_electricity production, oil_use
    - market for water, completely softened
    - market for water, decarbonised
    - offsite_market for municipal solid waste
    - offsite_market for municipal solid waste
    - offsite_market for municipal solid waste
    - offsite_market for municipal so

In [102]:
# Dictionnaire contenant les critères spécifiques pour chaque phase
phase_criteria = {
    'prod': {
        'include': [],  # Critères à inclure
        'exclude': ['diesel, burned in building machine', 'excavation']  # Critères à exclure
    },
    'dist': {
        'include': ['diesel, burned in building machine', 'excavation'],  # Aucun critère à inclure
        'exclude': []  # Critère à exclure
    },
    'use': {
        'include': [],  # Critère à inclure
        'exclude': ['market for photovoltaic facade installation, 3kWp, multi-Si, panel, mounted, at building', 
                    'market for diesel-electric generating set, 10MW', 'market for ammonia, anhydrous, liquid',
                    'market for heavy fuel oil','market for limestone, crushed, washed',
                    'market for oil power plant, 500MW','market for chimney',
                    'market for oil boiler, 100kW','market for oil storage, 3000l', 
                    'market for furnace, pellets, with silo, 300kW']  # Aucun critère à exclure
    },
    'eol': {
        'include': ['municipal solid waste','waste plastic, mixture', 'waste polystyrene', 'waste polyvinylchloride', 'waste polyethylene'],  # Aucun critère à inclure
        'exclude': []  # Aucun critère à exclure
    }
}

scenario_activities = [act for act in elec_bl if 'phase' in act['name']]

# Étape 1: Itérer sur les activités scénario-localisation-phase
for activity in scenario_activities:
    # Déterminer la phase pour chaque activité
    if 'production phase' in activity['name']:
        phase = 'prod'
    elif 'distribution phase' in activity['name']:
        phase = 'dist'
    elif 'use phase' in activity['name']:
        phase = 'use'
    elif 'end of life phase' in activity['name']:
        phase = 'eol'
    else:
        continue

    # Étape 2: Itérer sur les exchanges de chaque activité
    for exchange in activity.exchanges():
        sub_activity = exchange.input
        
        # Créer un nouveau nom avec un suffixe dépendant de la phase
        new_activity_name = sub_activity['name'] + f"_{phase}"
        new_activity = elec_bl.new_activity(code=generate_unique_id('ph'), name=new_activity_name)
        new_activity.save()

        # Étape 3: Ajouter des exchanges aux nouvelles activités selon les critères spécifiques à la phase
        for sub_exc in sub_activity.exchanges():
            # Vérifier les critères positifs (include) et négatifs (exclude)
            include_check = not phase_criteria[phase]['include'] or any(criterion in sub_exc.input['name'] for criterion in phase_criteria[phase]['include'])
            exclude_check = not phase_criteria[phase]['exclude'] or not any(criterion in sub_exc.input['name'] for criterion in phase_criteria[phase]['exclude'])

            if include_check and exclude_check:
                new_exchange = new_activity.new_exchange(
                    amount=sub_exc['amount'],
                    input=sub_exc.input,
                    type=sub_exc['type'],
                    unit=sub_exc['unit']
                )
                new_exchange.save()

        # Étape 4: Remplacer la sous-activité par la nouvelle activité dans l'exchange parent
        exchange.input = new_activity
        exchange.save()


In [17]:
# Remplacer les exchanges par ceux présents dans ma bdd

# Dictionnaire pour stocker les rapports de remplacement
replacement_report = {}

# Itérer à travers toutes les activités de la base de données
for activity in elec_bl:
    # Extraire la particule de l'activité actuelle
    activity_prefix = activity['name'].split('_')[0] + '_'
    
    # Créer une liste pour stocker les remplacements de cette activité
    replacement_report[activity['name']] = []

    # Itérer à travers les échanges de l'activité actuelle
    for exc in activity.exchanges():
        # Extraire le nom de base de l'échange cible (sans particule)
        base_name = exc.input['name']

        # Construire le nouveau nom de l'activité cible avec la particule de l'activité actuelle
        new_activity_name = activity_prefix + base_name

        # Chercher l'activité correspondante dans la base de données
        try:
            new_input = [act for act in elec_bl if act['name'] == new_activity_name][0]
            old_input_name = exc.input['name']
            exc['input'] = new_input.key  # Met à jour la référence de l'échange
            exc.save()  # Enregistre l'échange mis à jour
            
            # Ajouter le remplacement au rapport
            replacement_report[activity['name']].append({
                'old_exchange': old_input_name,
                'new_activity': new_activity_name
            })
        except :
            pass
    activity.save()
    
# Afficher le rapport de remplacement
for activity_name, replacements in replacement_report.items():
    if replacements:  # Afficher uniquement si des remplacements ont été effectués
        print(f"Activity: {activity_name}")
        for replacement in replacements:
            print(f"  - Replaced '{replacement['old_exchange']}' with '{replacement['new_activity']}'")
len(elec_bl)

Activity: waste_electricity production, oil
  - Replaced 'market for municipal solid waste' with 'waste_market for municipal solid waste'
Activity: offsite_excavation, hydraulic digger
  - Replaced 'market for diesel' with 'offsite_market for diesel'
  - Replaced 'market for lubricating oil' with 'offsite_market for lubricating oil'
Activity: offsite_diesel, burned in diesel-electric generating set, 10MW
  - Replaced 'market for lubricating oil' with 'offsite_market for lubricating oil'
Activity: onsite_diesel, burned in building machine
  - Replaced 'market for lubricating oil' with 'onsite_market for lubricating oil'
Activity: waste_inverter production, 2.5kW
  - Replaced 'market for waste polyethylene' with 'waste_market for waste polyethylene'
  - Replaced 'market for waste polystyrene' with 'waste_market for waste polystyrene'
Activity: onsite_diesel, burned in diesel-electric generating set, 10MW
  - Replaced 'market for lubricating oil' with 'onsite_market for lubricating oil'
A

120

In [25]:
# Trouver une liste d'activités et lister les échanges de chaque activité

# Step 1: Find all activities with 'phase' in their name
phase_activities = [act for act in elec_bl if 'phase' in act['name']]

for activity in phase_activities:
    print(f"Activity: {activity['name']}")
    for exc in activity.exchanges():
        print(f"  - {exc.input['name']}")           


Activity: offsite_electricity, baseline, distribution phase
  - offsite_distribution network construction, electricity, low voltage
  - offsite_oil power plant construction, 500MW
Activity: offsite_electricity, alternative, production phase
  - offsite_distribution network construction, electricity, low voltage
  - offsite_oil power plant construction, 500MW
  - offsite_photovoltaic facade installation, 3kWp, multi-Si, panel, mounted, at building
Activity: offsite_heat, alternative, end of life phase
  - offsite_furnace production, pellets, with silo, 300kW
  - offsite_hot water tank production, 600l
Activity: offsite_electricity, baseline, use phase
  - offsite_diesel, burned in diesel-electric generating set, 10MW
  - offsite_electricity production, oil
Activity: onsite_heat, alternative, use phase
  - onsite_heat production, light fuel oil, at boiler 100kW, non-modulating
Activity: onsite_heat, baseline, production phase
  - onsite_chimney production
  - onsite_oil boiler production

In [106]:
test = [
    activity for activity in eidb
    if 'market for municipal solid waste' in activity['name']    
    and 'RoW' in activity['location']  
][0]
test

'market for municipal solid waste' (kilogram, RoW, None)

In [123]:
# Checker les échanges dans une activité
for exc in test.technosphere():
    print(exc.input, exc.amount, exc.output)

'treatment of municipal solid waste, unsanitary landfill, dry infiltration class (100mm)' (kilogram, GLO, None) -0.0655322348590512 'market for municipal solid waste' (kilogram, RoW, None)
'treatment of municipal solid waste, unsanitary landfill, wet infiltration class (500mm)' (kilogram, GLO, None) -0.238658368530976 'market for municipal solid waste' (kilogram, RoW, None)
'treatment of municipal solid waste, open dump, wet infiltration class (500mm)' (kilogram, GLO, None) -0.207390654443263 'market for municipal solid waste' (kilogram, RoW, None)
'treatment of municipal solid waste, open dump, very wet infiltration class (1000mm)' (kilogram, GLO, None) -0.0319643984297913 'market for municipal solid waste' (kilogram, RoW, None)
'treatment of municipal solid waste, unsanitary landfill, hyperarid infiltration class (-250mm)' (kilogram, GLO, None) -0.0275828515230707 'market for municipal solid waste' (kilogram, RoW, None)
'treatment of municipal solid waste, unsanitary landfill, very w

In [29]:
# Supprimer toutes les activités répondant à un critère de la base de données
for activity in elec_bl:
    if ', onsite' in activity['name'] or ', offsite' in activity['name']:
#        activity.delete()
    else:
        pass

In [80]:
# Trouver une liste d'activités et lister les échanges de chaque activité

# Step 1: Find all activities with 'waste_' in their name
waste_activities = [act for act in elec_bl if 'waste_' in act['name']]

# Step 2: Create a dictionary to map exchanges to the activities where they are found
exchange_to_activities = {}

for activity in waste_activities:
    for exc in activity.exchanges():
        exchange_name = exc.input['name']
        if exchange_name not in exchange_to_activities:
            # Initialize a list for the exchange if it's not already in the dictionary
            exchange_to_activities[exchange_name] = []
        # Add the activity name to the list for this exchange
        exchange_to_activities[exchange_name].append(activity['name'])

# Display the exchanges along with the list of activities in which they are found
for exchange_name, activities in exchange_to_activities.items():
    print(f"Exchange: {exchange_name}, Found in activities: {', '.join(activities)}")

Exchange: waste_market for municipal solid waste, Found in activities: waste_excavation, hydraulic digger, waste_furnace production, pellets, with silo, 300kW, waste_furnace production, pellets, with silo, 300kW, waste_furnace production, pellets, with silo, 300kW, waste_furnace production, pellets, with silo, 300kW, waste_furnace production, pellets, with silo, 300kW, waste_furnace production, pellets, with silo, 300kW, waste_furnace production, pellets, with silo, 300kW, waste_furnace production, pellets, with silo, 300kW, waste_furnace production, pellets, with silo, 300kW, waste_excavation, skid-steer loader, waste_transport, freight, lorry >32 metric ton, EURO3, waste_transport, freight, lorry >32 metric ton, EURO3, waste_transport, freight, lorry >32 metric ton, EURO3, waste_oil storage production, 3000l, waste_oil storage production, 3000l, waste_oil storage production, 3000l, waste_road construction, waste_road construction, waste_photovoltaic mounting system production, for fa

In [80]:
# Sélectionner une activité dans une bdd
activity_1 = [
    activity for activity in elec_bl
    if 'offsite' in activity['name']
    and 'transport' in activity['name']   
#    and activity['location'] == 'RoW'
][0]
activity_1

'offsite_transport, freight, lorry >32 metric ton, EURO3' (ton kilometer, GLO, None)

In [81]:
for exc in activity_1.technosphere():
    print(exc.input['name'], exc['amount'])

market for diesel, low-sulfur 0.0195949765528
market for lorry, 40 metric ton 9.65e-08
market for maintenance, lorry 40 metric ton 9.65e-08
market for road 0.00109
market for road maintenance 8.09172e-05
offsite_market for municipal solid waste -1.52615789473684e-05
offsite_market for municipal solid waste -1.3140350877193e-05
offsite_market for municipal solid waste -0.000151207894736842


In [84]:
# Sélectionner une activité dans une bdd
activity_2 = [
    activity for activity in elec_bl
    if 'offsite' in activity['name']
    and 'burned' in activity['name']   
#    and activity['location'] == 'RoW'
][0]
activity_2

'offsite_diesel, burned in diesel-electric generating set, 10MW' (megajoule, GLO, None)

In [85]:
for exc in activity_2.technosphere():
    print(exc.input['name'], exc['amount'])

market group for diesel 0.0234
market for diesel-electric generating set, 10MW 1.85e-10
market for lubricating oil 6.700000000000002e-05
offsite_market for municipal solid waste -6.7e-05


In [86]:
# Sélectionner une activité dans une bdd
activity_3 = [
    activity for activity in elec_bl
    if 'offsite' in activity['name']
    and 'excavation' in activity['name']   
#    and activity['location'] == 'RoW'
][0]
activity_3

'offsite_excavation, skid-steer loader' (cubic meter, GLO, None)

In [87]:
for exc in activity_3.technosphere():
    print(exc.input['name'], exc['amount'])

market for building machine 9.09e-07
market for diesel 0.13
market for lubricating oil 0.003
offsite_market for municipal solid waste -0.003


In [90]:
# Sélectionner une activité dans une bdd
activity_4 = [
    activity for activity in elec_bl
    if 'offsite' in activity['name']
    and 'excavation' in activity['name']   
#    and activity['location'] == 'RoW'
][1]
activity_4

'offsite_excavation, hydraulic digger' (cubic meter, GLO, None)

In [91]:
for exc in activity_4.technosphere():
    print(exc.input['name'], exc['amount'])

market for diesel 0.131
market for hydraulic digger 1e-06
market for lubricating oil 0.0025
offsite_market for municipal solid waste -0.0025


In [111]:
# Sélectionner une activité dans une bdd
activity_6 = [
    activity for activity in eidb
    if 'diesel, burned in diesel-electric generating set, 10MW' in activity['name']
#    and 'road' in activity['name']   
#    and activity['location'] == 'RoW'
][0]
activity_6

'diesel, burned in diesel-electric generating set, 10MW, for oil and gas extraction' (megajoule, GLO, None)

In [112]:
for exc in activity_6.technosphere():
    print(exc.input['name'], exc['amount'])

market group for diesel 0.0234000001
market for diesel-electric generating set, 10MW 1.85000001e-10
market for lubricating oil 1.1658000017399198e-05
market for lubricating oil 5.53420000826008e-05
market for waste mineral oil -4.815380165393589e-05
market for waste mineral oil -1.839091251735534e-05
market for waste mineral oil -4.5528583870877175e-07


In [106]:
# Sélectionner une activité dans une bdd
activity_5 = [
    activity for activity in eidb
    if 'electricity production' in activity['name']
    and 'oil' in activity['name']   
    and activity['location'] == 'CA-NU'
][0]
activity_5

'electricity production, oil' (kilowatt hour, CA-NU, None)

In [107]:
for exc in activity_5.technosphere():
    print(exc.input['name'], exc['amount'])

market for ammonia, anhydrous, liquid 0.000749716483861458
market for bilge oil -2.53622741516298e-05
market for fly ash and scrubber sludge -3.13826647401871e-05
market for heavy fuel oil 0.289301745386436
market for lignite ash -0.000673476292640615
market for limestone, crushed, washed 0.0163237639637081
market for municipal solid waste -0.000178579718058257
market for oil power plant, 500MW 1.46595988103087e-11
market for residue from cooling tower -0.000161318814201025
market for waste mineral oil -0.000650001808939596
market for water, completely softened 0.120442704519077
market for water, decarbonised 0.599949098830776
